In [1]:
from django import setup
setup()

In [2]:
from django.utils import timezone
from entries.models import Entry, Nutrient

In [62]:
e1 = Entry(when=timezone.now(), what="something 4", quantity=2, category="c", measure="g")

In [4]:
e1.save()
e1.id

126

In [5]:
i1 = Nutrient()

In [6]:
i1.label = "sugar"
i1.unit = "g"
i1.category = "i"
i1.quantity = 4

In [3]:
from django.db.models import Sum, Count
Nutrient.objects.filter(category="o").aggregate(Sum("quantity"))

{'quantity__sum': 132.0}

## Reporting

In [4]:
from django.db.models.functions import TruncDay, TruncDate
from django.db.models import Sum, Count

In [5]:
import arrow
start_date = arrow.get("2016-11-16")
end_date = arrow.get("2016-12-06 23:59:59")
print(start_date, end_date)

2016-11-16T00:00:00+00:00 2016-12-06T23:59:59+00:00


In [ ]:
e1 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.FOOD_CONSUMPTION, 
    measure="g")
e1.save()

e2 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.PHYSICAL_ACTIVITY, 
    measure="g")

e2.save()

In [ ]:
e1n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.INTAKE,
    label="Energy",
    unit="kcal",
    quantity=140.5
)

e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="Energy",
    unit="kcal",
    quantity=10.5
)



In [6]:
Nutrient.get_energy_report(
    Nutrient.CATEGORIES.INTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(arrow.utcnow().naive))

{datetime.date(2016, 11, 16): 0,
 datetime.date(2016, 11, 17): 0,
 datetime.date(2016, 11, 18): 0,
 datetime.date(2016, 11, 19): 0,
 datetime.date(2016, 11, 20): 0,
 datetime.date(2016, 11, 21): 0,
 datetime.date(2016, 11, 22): 0,
 datetime.date(2016, 11, 23): 0,
 datetime.date(2016, 11, 24): 0,
 datetime.date(2016, 11, 25): 0,
 datetime.date(2016, 11, 26): 0,
 datetime.date(2016, 11, 27): 0,
 datetime.date(2016, 11, 28): 0,
 datetime.date(2016, 11, 29): 0,
 datetime.date(2016, 11, 30): 90.0,
 datetime.date(2016, 12, 1): 28.0,
 datetime.date(2016, 12, 2): 0,
 datetime.date(2016, 12, 3): 0,
 datetime.date(2016, 12, 4): 0,
 datetime.date(2016, 12, 5): 0,
 datetime.date(2016, 12, 6): 0,
 datetime.date(2016, 12, 7): 0}

In [59]:
report = Nutrient.get_energy_report(
    Nutrient.CATEGORIES.OUTTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(end_date.naive))

sorted(report.items())

[(datetime.date(2016, 11, 16), 0),
 (datetime.date(2016, 11, 17), 0),
 (datetime.date(2016, 11, 18), 0),
 (datetime.date(2016, 11, 19), 0),
 (datetime.date(2016, 11, 20), 0),
 (datetime.date(2016, 11, 21), 0),
 (datetime.date(2016, 11, 22), 0),
 (datetime.date(2016, 11, 23), 0),
 (datetime.date(2016, 11, 24), 0),
 (datetime.date(2016, 11, 25), 0),
 (datetime.date(2016, 11, 26), 0),
 (datetime.date(2016, 11, 27), 0),
 (datetime.date(2016, 11, 28), 0),
 (datetime.date(2016, 11, 29), 0),
 (datetime.date(2016, 11, 30), 0),
 (datetime.date(2016, 12, 1), 0),
 (datetime.date(2016, 12, 2), 1.32),
 (datetime.date(2016, 12, 3), 0),
 (datetime.date(2016, 12, 4), 0),
 (datetime.date(2016, 12, 5), 0),
 (datetime.date(2016, 12, 6), 12.5)]

In [38]:
a = (end_date - start_date)

In [52]:
records = Nutrient.objects.values("label", "unit").annotate(value=Sum("quantity"))
print(records.query.__str__())
for i in records:
    print(i)


SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"
{'value': 13.82, 'unit': u'kcal', 'label': u'Energy'}
{'value': 8.0, 'unit': u'mg', 'label': u'fat'}


In [24]:
e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="fat",
    unit="mg",
    quantity=3
)


In [43]:
nutrients = Nutrient.objects.filter(
        category="o",
        entry__when__range=[timezone.make_aware(start_date.naive), 
            timezone.make_aware(end_date.naive)]
    ).values(
        "label", "unit"
    ).annotate(
        value=Sum("quantity")
    )

In [42]:
nutrients.query.__str__()

u'SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" INNER JOIN "entries_entry" ON ("entries_nutrient"."entry_id" = "entries_entry"."id") WHERE ("entries_nutrient"."category" = o AND "entries_entry"."when" BETWEEN 2016-11-16 00:00:00 AND 2016-12-06 23:59:59) GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"'

In [13]:
nutrients = Nutrient.get_nutrients_report(
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(end_date.naive))

for i in nutrients:
    print(i)

{'value': 6.2, 'unit': u'mg', 'label': u'(+)-Catechin'}
{'value': 0.001, 'unit': u'g', 'label': u'10:0'}
{'value': 0.002, 'unit': u'g', 'label': u'12:0'}
{'value': 0.002, 'unit': u'g', 'label': u'14:0'}
{'value': 0.128, 'unit': u'g', 'label': u'16:0'}
{'value': 0.011, 'unit': u'g', 'label': u'16:1 undifferentiated'}
{'value': 0.013999999999999999, 'unit': u'g', 'label': u'18:0'}
{'value': 0.057999999999999996, 'unit': u'g', 'label': u'18:1 undifferentiated'}
{'value': 0.14200000000000002, 'unit': u'g', 'label': u'18:2 undifferentiated'}
{'value': 0.031, 'unit': u'g', 'label': u'18:3 undifferentiated'}
{'value': 0.082, 'unit': u'g', 'label': u'Alanine'}
{'value': 0.08499999999999999, 'unit': u'g', 'label': u'Arginine'}
{'value': 1.45, 'unit': u'g', 'label': u'Ash'}
{'value': 0.32899999999999996, 'unit': u'g', 'label': u'Aspartic acid'}
{'value': 0.1, 'unit': u'mg', 'label': u'Betaine'}
{'value': 21.0, 'unit': u'mg', 'label': u'Calcium, Ca'}
{'value': 29.34, 'unit': u'g', 'label': u'Carb

### Recommended Nutrients

In [23]:
import pandas as pd

In [46]:
pd.read_csv("/Users/muatik/Downloads/nutrients.csv", header=None)

,0,1,2,3,4,5
0,Total fat,g,65,Total lipid (fat),Proximates,g
1,Saturated fatty acids,g,20,"Fatty acids, total saturated",Lipids,g
2,Cholesterol,mg,300,Cholesterol,Proximates,mg
3,Sodium,mg,"2,400","Sodium, Na",Minerals,mg
4,Potassium,mg,"3,500","Potassium, K",Minerals,mg
5,Total carbohydrate,g,300,"Carbohydrate, by difference",Proximates,g
6,Fiber,g,25,"Fiber, total dietary",Proximates,g
7,Protein,g,50,Protein,Proximates,g
8,Vitamin A,IU,"5,000","Vitamin A, IU",Vitamins,IU
9,Vitamin C,mg,60,"Vitamin C, total ascorbic acid",Vitamins,mg


In [30]:
d = pd.read_csv("/Users/muatik/Downloads/nutrients.csv", header=None)
d = d.to_dict()

In [44]:
data = []
for i in range(len(d[0])):
    data.append({
        "label": d[0][i], 
        "unit": d[1][i], 
        "amount": d[2][i], 
        "usda_label": d[3][i], 
        "usda_category": d[4][i]})

In [42]:
data = {
    "headers": [
        "label",
        "unit",
        "amount",
        "usda_label",
        "usda_category"
    ],
    "labels": data
}

In [45]:
data

[{'amount': '65',
  'label': 'Total fat',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Total lipid (fat)'},
 {'amount': '20',
  'label': 'Saturated fatty acids',
  'unit': 'g',
  'usda_category': 'Lipids',
  'usda_label': 'Fatty acids, total saturated'},
 {'amount': '300',
  'label': 'Cholesterol',
  'unit': 'mg',
  'usda_category': 'Proximates',
  'usda_label': 'Cholesterol'},
 {'amount': '2,400',
  'label': 'Sodium',
  'unit': 'mg',
  'usda_category': 'Minerals',
  'usda_label': 'Sodium, Na'},
 {'amount': '3,500',
  'label': 'Potassium',
  'unit': 'mg',
  'usda_category': 'Minerals',
  'usda_label': 'Potassium, K'},
 {'amount': '300',
  'label': 'Total carbohydrate',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Carbohydrate, by difference'},
 {'amount': '25',
  'label': 'Fiber',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Fiber, total dietary'},
 {'amount': '50',
  'label': 'Protein',
  'unit': 'g',
  'usda_category': 'Prox

## User management

In [5]:
from django.contrib.auth.models import User
from entries.models import UserProfile

In [32]:
u = User(email="muatik@gmail.com", first_name="mustafa")
u.save()


In [58]:
u.userprofile = UserProfile(height=176, birth_date=timezone.make_aware(arrow.utcnow().naive))
u.userprofile.gender = UserProfile.MALE
u.userprofile.save()

In [59]:
u.last_name = "atik"
u.save()

In [6]:
u = User.objects.all()[0]

In [8]:
q, d = Nutrient.get_energy_report(u, "i", 
                        timezone.make_aware(start_date.naive), 
                        timezone.make_aware(end_date.naive))

In [9]:
q.query.__str__()

u'SELECT "entries_entry"."when", django_datetime_cast_date("entries_entry"."when", UTC) AS "date", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" INNER JOIN "entries_entry" ON ("entries_nutrient"."entry_id" = "entries_entry"."id") WHERE ("entries_nutrient"."category" = i AND "entries_entry"."user_id" = 1 AND "entries_entry"."when" BETWEEN 2016-11-16 00:00:00 AND 2016-12-06 23:59:59 AND "entries_nutrient"."unit" = kcal AND "entries_nutrient"."label" = Energy) GROUP BY django_datetime_cast_date("entries_entry"."when", UTC), "entries_entry"."when"'

In [23]:
Nutrient.objects.filter(
            entry=Entry.objects.get(id=33),
            label="Energy",
            unit="kcal",
            category=Nutrient.CATEGORIES.OUTTAKE
        ).aggregate(Sum("quantity"))

{'quantity__sum': None}

[]